<div style="text-align: right">INFO 6210 Data Management and Database Design </div>
<div style="text-align: right"> Suman Rawat : NUID 001058600</div>
<div style="text-align: right"> Shakti Chetan : NUID 001375816</div>

# Assignment 2 - Physical Data Model and Normalization

## Abstract

In this assignment, we are assumed to be working for a company called Nerd Analytics and that you are completely in charge of the database. We will create a physical database, normalize, actively stream social media data to the database we created in Assignment 1.

We have taken new dataset related to NBA Game. This dataset contains 5 files:

1. games: all games from 2004 season to last update with the date, teams and some details like number of points, etc.
2. games_details : details of games dataset, all statistics of players for a given game
3. players : players details (name)
4. teams : all teams of NBA


# Conceptual Diagram

<br>
<center>
<img src = "Conceptual_Diagram.png" width = "400" >
</center>

# ER Diagram

<br>
<center>
<img src = "ER_Diagram.png" width = "400" >
</center>

# Physical Diagram

<br>
<center>
<img src = "PhysicalDiagram.jpeg" width = "400" >
</center>

## 1. Extracting data from csv files

In [88]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests
import lxml.html as lh
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rawg import Rawg
import pandas.io.json as pd_json
import rawgpy

# What is Normalization?

Normalization is a database design technique which organizes tables in a manner that reduces redundancy and dependency of data. It divides larger tables to smaller tables and links them using relationships. 

## 1NF (First Normal Form) Rules
1. Each table has a primary key: minimal set of attributes which can uniquely identify a record.
2. The values in each column of a table are atomic (No multi-value attributes allowed).
3. There are no repeating groups: two columns do not store similar information in the same table. 

## 2NF (Second Normal Form) Rules
1. All requirements for 1st NF must be met.
2. No partial dependencies.
3. No calculated data 

## 3NF (Third Normal Form) Rules

1. All requirements for 2nd NF must be met.
2. Eliminate fields that do not directly depend on the primary key; that is no transitive dependencies. 
 

# Step 1: Game_Details Table - Importing data from csv file

In [126]:
gameDetailCSV = pd.read_csv(r"C:\Users\ankur\Downloads\DMDD Notebooks\games_Details.csv")
gameDetailCSV

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,21900741,1610612753,ORL,Orlando,1628411,Wes Iwundu,F,NaN,22:14,3.0,...,0.0,2.0,2.0,2.0,1.0,0.0,0.0,1.0,9.0,2.0
1,21900741,1610612753,ORL,Orlando,203932,Aaron Gordon,F,NaN,37:39,6.0,...,2.0,10.0,12.0,5.0,2.0,0.0,1.0,2.0,16.0,13.0
2,21900741,1610612753,ORL,Orlando,202696,Nikola Vucevic,C,NaN,36:27,10.0,...,3.0,4.0,7.0,5.0,0.0,0.0,1.0,1.0,22.0,11.0
3,21900741,1610612753,ORL,Orlando,203095,Evan Fournier,G,NaN,30:49,7.0,...,0.0,3.0,3.0,1.0,0.0,0.0,3.0,1.0,17.0,23.0
4,21900741,1610612753,ORL,Orlando,1628365,Markelle Fultz,G,NaN,34:42,4.0,...,0.0,1.0,1.0,14.0,2.0,0.0,2.0,3.0,12.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572998,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,19,4.0,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
572999,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,23,7.0,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
573000,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,15,3.0,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
573001,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,19,1.0,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


### Table is in 0NF

In [127]:
# Checking the number of rows and columns in dataframe
print('Number of Rows and Columns are : ',gameDetailCSV.shape)

Number of Rows and Columns are :  (573003, 28)


In [128]:
gameDetailCSV.dtypes

GAME_ID                int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_CITY             object
PLAYER_ID              int64
PLAYER_NAME           object
START_POSITION        object
COMMENT               object
MIN                   object
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TO                   float64
PF                   float64
PTS                  float64
PLUS_MINUS           float64
dtype: object

In [130]:
# Removing Unwanted Columns from the dataset
gameDetailDF = gameDetailCSV.iloc[:, [0,1,2,3,4,5,26]]

In [131]:
gameDetailDF.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,PTS
0,21900741,1610612753,ORL,Orlando,1628411,Wes Iwundu,9.0
1,21900741,1610612753,ORL,Orlando,203932,Aaron Gordon,16.0
2,21900741,1610612753,ORL,Orlando,202696,Nikola Vucevic,22.0
3,21900741,1610612753,ORL,Orlando,203095,Evan Fournier,17.0
4,21900741,1610612753,ORL,Orlando,1628365,Markelle Fultz,12.0


Since dataframe has no single unique primary key we will find the combination of keys to find the composite key for the dataset.

In [132]:

from itertools import combinations, chain

full_list = chain.from_iterable(combinations(gameDetailDF, i) for i in range(1, len(gameDetailDF.columns)+1))

n = len(gameDetailDF.index)

res = []
for cols in full_list:
    cols = list(cols)
    if len(gameDetailDF[cols].drop_duplicates().index) == n:
        res.append(cols)

print(len(res))  # 64 combination for composite keys

32


### List of Composite keys

In [133]:
print(res)

[['GAME_ID', 'PLAYER_ID'], ['GAME_ID', 'TEAM_ID', 'PLAYER_ID'], ['GAME_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID'], ['GAME_ID', 'TEAM_CITY', 'PLAYER_ID'], ['GAME_ID', 'PLAYER_ID', 'PLAYER_NAME'], ['GAME_ID', 'PLAYER_ID', 'PTS'], ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID'], ['GAME_ID', 'TEAM_ID', 'TEAM_CITY', 'PLAYER_ID'], ['GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'PLAYER_NAME'], ['GAME_ID', 'TEAM_ID', 'PLAYER_ID', 'PTS'], ['GAME_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID'], ['GAME_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME'], ['GAME_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PTS'], ['GAME_ID', 'TEAM_CITY', 'PLAYER_ID', 'PLAYER_NAME'], ['GAME_ID', 'TEAM_CITY', 'PLAYER_ID', 'PTS'], ['GAME_ID', 'PLAYER_ID', 'PLAYER_NAME', 'PTS'], ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID'], ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PLAYER_NAME'], ['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_ID', 'PTS'], ['GAME_ID', 'TEAM_ID', 'TEAM_CITY', 'PL

### Adding one column for the primary key.


In [134]:

gameDetailDF['ID'] = range(1, 1+len(gameDetailDF))

C:\Users\ankur\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [135]:
gameDetailDF.head() # Primary key is ID

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,PTS,ID
0,21900741,1610612753,ORL,Orlando,1628411,Wes Iwundu,9.0,1
1,21900741,1610612753,ORL,Orlando,203932,Aaron Gordon,16.0,2
2,21900741,1610612753,ORL,Orlando,202696,Nikola Vucevic,22.0,3
3,21900741,1610612753,ORL,Orlando,203095,Evan Fournier,17.0,4
4,21900741,1610612753,ORL,Orlando,1628365,Markelle Fultz,12.0,5


## Step 2: Converting game_details table into 2NF

### Break the games_details table into player and team tables to remove the partial dependecies

### A. Creating tables for Players Details

In [140]:
# Removing PLAYER_ID ,PLAYER_NAME, TEAM_ID creating new Player Table

playerTable = gameDetailDF.iloc[:,[4,5,1]]

In [141]:
playerTable.shape

(573003, 3)

In [142]:
#nullValueDF = gamesCSV.GAME_ID.notnull()
playerTable = playerTable.drop_duplicates(subset=['PLAYER_ID','PLAYER_NAME','TEAM_ID'], keep='first')

In [147]:
playerTable.PLAYER_ID.nunique(dropna = True)

2323

In [148]:
 playerTable.sort_values(['TEAM_ID'], axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

,PLAYER_ID,PLAYER_NAME,TEAM_ID
501494,201858,Cartier Martin,1610612737
388049,203098,John Jenkins,1610612737
388058,1628535,Jordan Mathews,1610612737
212449,2215,Jason Collins,1610612737
252670,202348,Jordan Crawford,1610612737
...,...,...,...
427063,203584,Troy Daniels,1610612766
427066,203268,Jorge Gutierrez,1610612766
56502,976,Jeff McInnis,1610612766
465464,203106,Jeffery Taylor,1610612766


### Since Player Table is not is 2NF we will converting it into 2 NF

In [149]:
playerTeamTable = playerTable.iloc[:,[0,1]]

In [150]:
playerTeamTable = playerTeamTable.drop_duplicates(subset=['PLAYER_ID','PLAYER_NAME'], keep='first')
playerTeamTable .head()

,PLAYER_ID,PLAYER_NAME
0,1628411,Wes Iwundu
1,203932,Aaron Gordon
2,202696,Nikola Vucevic
3,203095,Evan Fournier
4,1628365,Markelle Fultz


### Player table is in 2NF. To change it into 3NF we will slice the table to remove transition dependencies

In [108]:
playerNteamTable = playerTable.iloc[:,[1,2]]
playerNteamTable = playerNteamTable.drop_duplicates(subset=['TEAM_ID','PLAYER_NAME'], keep='first')

In [109]:
playerNteamTable.head()

,PLAYER_NAME,TEAM_ID
0,Wes Iwundu,1610612753
1,Aaron Gordon,1610612753
2,Nikola Vucevic,1610612753
3,Evan Fournier,1610612753
4,Markelle Fultz,1610612753


## 3NF
1. All requirements for 2nd NF must be met.
2. Eliminate fields that do not directly depend on the primary key; that is no transitive dependencies. 

Hence Player table is in 3NF

### B. Creating Table for Team details

In [151]:
teamTable = gameDetailDF.iloc[:,[0,1,2,3]]

In [152]:
teamTable.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY
0,21900741,1610612753,ORL,Orlando
1,21900741,1610612753,ORL,Orlando
2,21900741,1610612753,ORL,Orlando
3,21900741,1610612753,ORL,Orlando
4,21900741,1610612753,ORL,Orlando


In [153]:
teamTable = teamTable.drop_duplicates(subset=['TEAM_ID'], keep='first')

In [154]:
 teamTable.sort_values(['TEAM_ID'], axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY
110,21900745,1610612737,ATL,Atlanta
97,21900745,1610612738,BOS,Boston
36,21900742,1610612739,CLE,Cleveland
274,21900738,1610612740,NOP,New Orleans
326,21900740,1610612741,CHI,Chicago
49,21900743,1610612742,DAL,Dallas
250,21900737,1610612743,DEN,Denver
74,21900744,1610612744,GSW,Golden State
287,21900738,1610612745,HOU,Houston
237,21900750,1610612746,LAC,LA


## Team Table is in 2NF
1. All requirements for 1st NF must be met.
2. No partial dependencies.
3. No calculated data 
 
Hence game_details table is now in 2NF

## 3NF
1. All requirements for 2nd NF must be met.
2. Eliminate fields that do not directly depend on the primary key; that is no transitive dependencies. 

Hence Team table is in 3NF as well

## Step 3 : Slicing the games details table will remove the partial dependencies. 


In [116]:
gameDetail2NFDF = gameDetailDF.iloc[:,[7,0,1,4,6]]

In [117]:
gameDetail2NFDF.head()

,ID,GAME_ID,TEAM_ID,PLAYER_ID,PTS
0,1,21900741,1610612753,1628411,9.0
1,2,21900741,1610612753,203932,16.0
2,3,21900741,1610612753,202696,22.0
3,4,21900741,1610612753,203095,17.0
4,5,21900741,1610612753,1628365,12.0


### Game_details is now in 2NF

# Step 4 : For table to be in 3NF :
1. All requirements for 2nd NF must be met.
2. Eliminate fields that do not directly depend on the primary key; that is no transitive dependencies. 

In [157]:
gameDetail3NFDF = gameDetail2NFDF.iloc[:,[0,1,3,4]]

In [158]:
gameDetail3NFDF = gameDetail3NFDF.drop_duplicates(subset=['GAME_ID','PLAYER_ID','PTS','ID'], keep='first')

In [159]:
gameDetail3NFDF.head()

,ID,GAME_ID,PLAYER_ID,PTS
0,1,21900741,1628411,9.0
1,2,21900741,203932,16.0
2,3,21900741,202696,22.0
3,4,21900741,203095,17.0
4,5,21900741,1628365,12.0


# Hence the games_detail table is in 3NF form now.

# Conclusion
In this assignment, we have create a physical database, normalize, actively stream social media data to the database. Another group of statisticians and machine learning experts will be using the data that we have modeled, gather, clean and database analyze Social Media for a NBA Games domain. For NBA game domain we have entities like games, player, teams. We have performed database normalization on the game_details table. Normalization involves refactoring a table into smaller (and less redundant) tables but without losing information; defining foreign keys in the old table referencing the primary keys of the new ones. The objective is to isolate data so that additions, deletions, and modifications of an attribute can be made in just one table and then propagated through the rest of the database using the defined foreign keys. We have checked whether our tables are in First normal form (1NF), Second normal form (2NF) and Third normal form (3NF).  If not, we have restructure the database so that all of the tables are in Third normal form; that is, we have normalized the database.

# References

1. NBA Dataset : https://www.kaggle.com/nathanlauga/nba-games#ranking.csv
2. ER Diagram : https://www.draw.io/
3. Normalization : https://www.studytonight.com/dbms/database-normalization.php
4. https://www.elstel.org/database/dbschemacmd.html.en
5. https://www.nba.com/

# Contribution

1. Conception or design of the work – Shakti and Suman
2. Data collection for Web scraping:  Suman
3. Data collection for web API and raw csv file: Shakti Chetan
4. Drafting the article: Suman
5. Creating Conceptual Schema: Shakti Chetan
6. Creation of ER Diagram : Suman Rawat
7. Creation of Physical Model: Shakti Chetan
6. Critical revision of the article:  Suman and Shakti
